# 1. Loading data


In [25]:
import pandas as pd


In [26]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

# brands.csv
url = "https://drive.google.com/file/d/1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD/view?usp=share_link"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)

# 2. Cleaning data

In [27]:
# checking datatypes
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


## 2.1 cleaning orders

#### change datatype 'created_date'

In [28]:
orders['created_date'] = pd.to_datetime(orders['created_date']) #change the type of the created date column.

In [29]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226909 non-null  int64         
 1   created_date  226909 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.9+ MB


#### change 'state' to 'status'

In [30]:
orders.columns=["order_id","created_date","total_paid","status"]#change state to status
orders

,order_id,created_date,total_paid,status
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled
...,...,...,...,...
226904,527397,2018-03-14 13:56:38,42.99,Place Order
226905,527398,2018-03-14 13:57:25,42.99,Shopping Basket
226906,527399,2018-03-14 13:57:34,141.58,Shopping Basket
226907,527400,2018-03-14 13:57:41,19.98,Shopping Basket


#### looking for missing values

In [31]:
orders.isna().sum() # looking for missing values

order_id        0
created_date    0
total_paid      5
status          0
dtype: int64

In [32]:
#drop the 5 nan values
orders.dropna(inplace=True)

In [33]:
orders.isna().sum() # looking for missing values

order_id        0
created_date    0
total_paid      0
status          0
dtype: int64

#### checking 'status' 'Place Order' --> same as shopping basket (check date and created date)

In [34]:
orders['status'].unique()

array(['Cancelled', 'Completed', 'Pending', 'Shopping Basket',
       'Place Order'], dtype=object)

In [35]:
orders_orderlines = orders.merge(
    orderlines,
    how="outer",
    left_on="order_id",
    right_on="id_order"
)
filtered_data = orders_orderlines.loc[orders_orderlines['status'] == "Place Order", ['order_id', 'created_date', 'date']].sort_values('order_id').head(10)
filtered_data
# date --> orderlines
# created_date -->orders

,order_id,created_date,date
306,299553.0,2017-01-01 02:14:47,2017-01-01 02:14:47
308,299555.0,2017-01-01 02:18:45,2017-01-01 02:18:45
335,299581.0,2017-01-01 08:52:28,2017-09-28 20:35:53
336,299581.0,2017-01-01 08:52:28,2017-09-29 06:56:54
337,299581.0,2017-01-01 08:52:28,2017-10-01 19:03:04
367,299602.0,2017-01-01 10:13:44,2017-01-01 10:13:44
371,299606.0,2017-01-01 10:40:34,2017-01-01 10:41:11
377,299611.0,2017-01-01 10:53:57,2017-01-01 10:53:57
380,299614.0,2017-01-01 10:58:31,2017-01-01 10:58:31
385,299617.0,2017-01-01 11:07:57,2017-01-01 11:07:57


In [36]:
orders_orderlines = orders.merge(
    orderlines,
    how="outer",
    left_on="order_id",
    right_on="id_order"
)
filtered_data = orders_orderlines.loc[orders_orderlines['status'] == "Shopping Basket", ['order_id', 'created_date', 'date']].sort_values('order_id').head(10)
filtered_data
# date --> orderlines
# created_date -->orders

,order_id,created_date,date
293,299539.0,2017-01-01 00:07:19,2017-01-01 00:07:19
294,299540.0,2017-01-01 00:19:45,2017-01-01 00:19:45
295,299541.0,2017-01-01 00:20:57,2017-01-01 00:20:57
296,299542.0,2017-01-01 00:51:40,2017-01-01 00:51:40
297,299543.0,2017-01-01 01:06:38,2017-01-01 01:06:38
298,299544.0,2017-01-01 01:17:21,2017-01-01 01:17:21
301,299548.0,2017-01-01 02:02:20,2017-01-01 02:02:20
303,299550.0,2017-01-01 02:09:52,2017-01-01 02:09:52
304,299552.0,2017-01-01 02:13:51,2017-01-01 02:13:51
305,299552.0,2017-01-01 02:13:51,2017-01-01 02:14:36


In [37]:
# every Place Order goes into shopping basket, they are same category
orders.replace("Place Order",
           "Shopping Basket",
           inplace=True)

In [38]:
orders['status'].unique()

array(['Cancelled', 'Completed', 'Pending', 'Shopping Basket'],
      dtype=object)

## 2.2 Cleaning orderlines

In [39]:
orderlines['date'] = pd.to_datetime(orderlines['date'])#changing the date from object to datetime.
orderlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 15.7+ MB


In [40]:
orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts()#how many value with 2 dots

False    257814
True      36169
Name: unit_price, dtype: int64

In [41]:
orderlines['unit_price'] = orderlines.unit_price.str.replace(r'(\d*)\.(\d+)\.(\d+)', r'\1\2.\3')#remove the first dot.

<ipython-input-41-a3a6ec28a0b8>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  orderlines['unit_price'] = orderlines.unit_price.str.replace(r'(\d*)\.(\d+)\.(\d+)', r'\1\2.\3')#remove the first dot.


In [42]:
orderlines.unit_price.str.contains("\d+\.\d+\.\d+").value_counts() #control if values with two dots are gone

False    293983
Name: unit_price, dtype: int64

In [43]:
orderlines.unit_price = pd.to_numeric(orderlines.unit_price)
orderlines["unit_price"].nlargest(10)#here we have just one price outlier so we will drop it.

36979     159989.83
38317      15349.00
282442     14725.00
232052     14580.00
232077     14580.00
234834     14580.00
241912     14580.00
248338     14580.00
253465     14580.00
255123     14580.00
Name: unit_price, dtype: float64

In [44]:
orderlines.drop(orderlines[orderlines.unit_price==159989.83].index, inplace=True)

In [45]:
orderlines["unit_price"].nlargest(10)

38317     15349.0
282442    14725.0
232052    14580.0
232077    14580.0
234834    14580.0
241912    14580.0
248338    14580.0
253465    14580.0
255123    14580.0
255454    14580.0
Name: unit_price, dtype: float64

In [46]:
orderlines["unit_price"].nsmallest(10)#we have one weird price (-119.0) so we drop it

77008   -119.0
53515      0.0
53530      0.0
56529      0.0
56562      0.0
56576      0.0
56617      0.0
56636      0.0
56641      0.0
56648      0.0
Name: unit_price, dtype: float64

In [47]:
orderlines.drop(orderlines[orderlines.unit_price==(-119.0)].index, inplace=True)# dropping the -119 value

In [48]:
orderlines["unit_price"].nsmallest(10)#checking if value is gone

53515    0.0
53530    0.0
56529    0.0
56562    0.0
56576    0.0
56617    0.0
56636    0.0
56641    0.0
56648    0.0
56665    0.0
Name: unit_price, dtype: float64

In [49]:
orderlines.drop("product_id",axis=1,inplace=True)#delete product_id column because it is not used any more
orderlines

,id,id_order,product_quantity,sku,unit_price,date
0,1119109,299539,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...
293978,1650199,527398,1,JBL0122,42.99,2018-03-14 13:57:25
293979,1650200,527399,1,PAC0653,141.58,2018-03-14 13:57:34
293980,1650201,527400,2,APP0698,9.99,2018-03-14 13:57:41
293981,1650202,527388,1,BEZ0204,19.99,2018-03-14 13:58:01


In [50]:
orderlines.duplicated().sum()# we don't have duplicates in orderlines.

0

In [51]:
orderlines.isna().sum()#we don't have any null values in orderlines.

id                  0
id_order            0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

In [52]:
orderlines.columns=["id","order_id","product_quantity","sku","unit_price","date"] #changing id_order to order_id

In [53]:
orderlines

,id,order_id,product_quantity,sku,unit_price,date
0,1119109,299539,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...
293978,1650199,527398,1,JBL0122,42.99,2018-03-14 13:57:25
293979,1650200,527399,1,PAC0653,141.58,2018-03-14 13:57:34
293980,1650201,527400,2,APP0698,9.99,2018-03-14 13:57:41
293981,1650202,527388,1,BEZ0204,19.99,2018-03-14 13:58:01


## 2.3 Cleaning products

In [54]:
products

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [55]:
products.drop_duplicates(inplace=True)#droping duplicates
products.shape #schows rows and columns
#(10580,7)

(10580, 7)

In [56]:
products.isna().sum()#checking for missing values

sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64

In [57]:
products.price.str.contains("\d+\.\d+\.\d+").value_counts() # Counting the number of values in the 'price' column that contain a string of digits separated by two dots.

False    10157
True       377
Name: price, dtype: int64

In [58]:
import numpy as np
products.loc[products["price"].str.contains("\d+\.\d+\.\d+")==True,"price"]=np.nan # Replacing any values in the 'price' column that contain the string pattern "\d+\.\d+\.\d+" with NaN (missing value)

#### Replacing null values with max price for a product

In [59]:
products["price"].isna().sum()#checking for missing values again

423

In [60]:
#defining a variables max prices, that we use later to fill in the missing prices
#using the max of unit_price as original price
max_prices=orderlines.groupby("sku").agg({"unit_price":"max"})
max_prices

,unit_price
sku,
8MO0001-A,12.96
8MO0003-A,12.85
8MO0007,23.99
8MO0008,23.99
8MO0009,23.99
...,...
ZAG0041,19.99
ZAG0042,19.99
ZEP0007,139.90


In [61]:
max_prices.isna().sum() #checking that there are no null values in max_prices

unit_price    0
dtype: int64

In [62]:
#all products (skus) that have null values to frame
nan_products=products.loc[products['price'].isna(),'sku']
nan_products.to_frame()

,sku
34,TWS0019
665,CRU0015-2
792,APP0672
797,APP0673
827,PAC0339
...,...
19312,REP0424
19313,REP0421
19314,REP0416
19315,REP0413


In [63]:
#merging max_prices to nan_products, so all products have prices now in this dataframe
df=max_prices.merge(nan_products,how="left",on="sku")
df


,sku,unit_price
0,8MO0001-A,12.96
1,8MO0003-A,12.85
2,8MO0007,23.99
3,8MO0008,23.99
4,8MO0009,23.99
...,...,...
7946,ZAG0041,19.99
7947,ZAG0042,19.99
7948,ZEP0007,139.90
7949,ZEP0008,104.12


In [64]:
df.isna().sum()#checking that no vull values are left

sku           0
unit_price    0
dtype: int64

In [65]:
#merging products with our new df
products=products.merge(df,how="left",on="sku")


In [66]:
# updating the missing values for unit price with price if there an none, otherwise unit_price stays as it is
products["unit_price"].update(products["price"])


In [67]:
products

,sku,name,desc,price,promo_price,in_stock,type,unit_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401,59
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387,59
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230,25
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364,34.99
...,...,...,...,...,...,...,...,...
10575,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282,29.99
10576,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,69.95
10577,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,69.95
10578,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392,69.95


In [68]:
products["unit_price"].isna().sum()

193

In [69]:
#dropping the few null values that are left
products.dropna(inplace=True)

In [70]:
products.isna().sum()

sku            0
name           0
desc           0
price          0
promo_price    0
in_stock       0
type           0
unit_price     0
dtype: int64

In [71]:
products=products.drop(['in_stock', 'price','promo_price'], axis=1)#drop usless columns

In [72]:
products.columns=["sku","name","desc","type","price"]#rename unit_price= price

In [73]:
products['price'] = pd.to_numeric(products['price']) #change datatype

In [74]:
products

,sku,name,desc,type,price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,59.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,1387,59.00
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,1230,25.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,1364,34.99
...,...,...,...,...,...
10575,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,12282,29.99
10576,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,1392,69.95
10577,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,1392,69.95
10578,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,1392,69.95


In [75]:
# adding short column to products to make it easier to connect to brands.csv
products['short'] = products['sku'].str[:3]
products.head(1)

,sku,name,desc,type,price,short
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,59.99,RAI


In [76]:
products

,sku,name,desc,type,price,short
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,59.99,RAI
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,59.00,APP
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,1387,59.00,APP
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,1230,25.00,APP
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,1364,34.99,KIN
...,...,...,...,...,...,...
10575,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,12282,29.99,BEL
10576,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,1392,69.95,THU
10577,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,1392,69.95,THU
10578,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,1392,69.95,THU


In [77]:
po=products.merge(orderlines,how="inner",on="sku")

In [78]:
df1=po[po["unit_price"]>po["price"]]["sku"]
len(df1)

11621

In [79]:
import numpy as np
products.loc[products["sku"].isin(df1),"price"]=np.nan

In [80]:
products["price"].isna().sum()

619

In [81]:
df1.to_frame()

,sku
329,APP0023
332,APP0023
333,APP0023
601,KIN0007
610,KIN0001-2
...,...
285094,BOS0067
285102,BOS0068
285104,BOS0068
285105,BOS0068


In [82]:
df2=max_prices.merge(df1,how="left",on="sku")

In [83]:
products=products.merge(df2,how="left",on="sku")

In [ ]:
products

,sku,name,desc,type,price,short,unit_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,59.99,RAI,54.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
2,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
3,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
4,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,1387,59.00,APP,59.00
...,...,...,...,...,...,...,...
21101,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,12282,29.99,BEL,NaN
21102,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,NaN
21103,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,NaN
21104,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,NaN


In [85]:
products["unit_price"].update(products["price"])

In [86]:
products

,sku,name,desc,type,price,short,unit_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,59.99,RAI,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
2,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
3,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,NaN,APP,60.49
4,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,1387,59.00,APP,59.00
...,...,...,...,...,...,...,...
21101,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,12282,29.99,BEL,29.99
21102,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,69.95
21103,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,69.95
21104,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,1392,69.95,THU,69.95


In [88]:
products["unit_price"].isna().sum()#checking that all prices are filled with max price

0

In [89]:
products=products.drop(['price'], axis=1)

In [90]:
products.columns=["sku","name","desc","type","short","price"]

In [91]:
products

,sku,name,desc,type,short,price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,8696,RAI,59.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,APP,60.49
2,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,APP,60.49
3,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,13855401,APP,60.49
4,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,1387,APP,59.00
...,...,...,...,...,...,...
21101,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,12282,BEL,29.99
21102,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,1392,THU,69.95
21103,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,1392,THU,69.95
21104,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,1392,THU,69.95


In [ ]:
# get cleaned data in new csv files

In [ ]:
from google.colab import drive

drive.mount('drive')

Mounted at drive


In [ ]:
orders.to_csv("/content/drive/MyDrive/orders_clean.csv", index=False)

In [ ]:
orderlines.to_csv("/content/drive/MyDrive/orderlines_clean.csv", index=False)

In [ ]:
products.to_csv("/content/drive/MyDrive/products_clean.csv", index=False)

In [ ]:
brands.to_csv("/content/drive/MyDrive/brands_clean.csv", index=False)